# Module 3: Periodic Action Module (UTH-Based Adjustments)

## Purpose
This module runs at 12 PM, 3 PM, 6 PM, 9 PM, and 12 AM Cairo time to:
1. Adjust prices based on Up-Till-Hour (UTH) performance vs benchmarks
2. Manage SKU discounts and Quantity Discounts based on performance
3. Adjust cart rules dynamically

## UTH Benchmarks
- Calculate historical qty from start of day till current hour over the last 4 months
- Multiply by P80 all-time-high quantity and P70 retailers

## Action Logic
- **On Track (±10%)**: No action
- **Growing (>110%)**: Deactivate discounts or increase price, reduce cart if too open
- **Dropping (<90%)**: Reduce price, increase cart by 20%
- **Zero Demand (qty=0 today)**: Market min + SKU discount


In [1]:
%%capture

# Upgrade pip
!pip install --upgrade pip
# Connectivity
!pip install psycopg2-binary
!pip install snowflake-connector-python==3.15.0
!pip install snowflake-sqlalchemy
!pip install warnings
!pip install keyring==23.11.0
!pip install sqlalchemy==1.4.46
!pip install requests
!pip install boto3
!pip install oauth2client
!pip install gspread==5.9.0
!pip install gspread_dataframe
!pip install google.cloud
# Data manipulation and analysis
!pip install polars
!pip install pandas==2.2.1
!pip install numpy
!pip install openpyxl
!pip install xlsxwriter
# Date and time handling
!pip install --upgrade datetime
!pip install python-time
!pip install --upgrade pytz
# Progress bar
!pip install tqdm
# Database data types
!pip install db-dtypes
# Modeling
!pip install statsmodels
!pip install scikit-learn
!pip install import-ipynb
# Plotting
!pip install matplotlib
!pip install seaborn

In [2]:
# =============================================================================
# IMPORTS AND SETUP
# =============================================================================
import pandas as pd
import numpy as np
import os
from datetime import datetime
import pytz
import sys
sys.path.append('..')

# Run queries_module - this:
# 1. Initializes Snowflake credentials (setup_environment_2.initialize_env())
# 2. Provides query_snowflake() function
# 3. Provides TIMEZONE from Snowflake
# 4. Provides get_current_stocks(), get_current_prices(), get_current_wac(), get_current_cart_rules()
%run queries_module.ipynb

# Run market_data_module - this:
# 1. Provides get_market_data() for fetching fresh market prices (NO INPUT REQUIRED)
# 2. Provides get_margin_tiers() for fetching margin tiers (NO INPUT REQUIRED)
# 3. Fetches Ben Soliman, Marketplace, and Scrapped prices
# 4. Fills missing prices from group-level data
# 5. Calculates market price percentiles and margin tiers
%run market_data_module.ipynb

# Cairo timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()
CURRENT_HOUR = CAIRO_NOW.hour

# Configuration
UTH_GROWING_THRESHOLD = 1.10    # >110% = Growing
UTH_DROPPING_THRESHOLD = 0.90   # <90% = Dropping
CART_INCREASE_PCT = 0.20        # 20% cart increase
CART_DECREASE_PCT = 0.20        # 20% cart decrease
MIN_CART_RULE = 5
MAX_CART_RULE = 150
MIN_PRICE_CHANGE_EGP = 0.25     # Minimum 0.25 EGP for any price change
CONTRIBUTION_THRESHOLD = 50     # 50% contribution threshold
MAX_PRICE_REDUCTIONS_PER_DAY = 2  # Max price reductions per day
# SKU discount percentage will be decided in sku_discount_handler

# Input/Output configuration
# Data is now loaded from Snowflake instead of Excel
INPUT_TABLE = 'MATERIALIZED_VIEWS.Pricing_data_extraction'
PREVIOUS_OUTPUT_TABLE = 'MATERIALIZED_VIEWS.pricing_periodic_push'
OUTPUT_FILE = f'module_3_output_{CAIRO_NOW.strftime("%Y%m%d_%H%M")}.xlsx'

print(f"Module 3: Periodic Actions")
print(f"Run Time (Cairo): {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Current Hour (Cairo): {CURRENT_HOUR}")
print(f"Input: {INPUT_TABLE} (today's data)")


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


Queries Module | Timezone: America/Los_Angeles
✅ UTH and Last Hour functions defined

QUERIES MODULE READY

Live Data Functions:
  • get_current_stocks()
  • get_packing_units()
  • get_current_prices()
  • get_current_wac()
  • get_current_cart_rules()

UTH Performance Functions:
  • get_uth_performance()         - UTH qty/retailers (Snowflake)
  • get_hourly_distribution()     - Historical hour contributions (Snowflake)
  • get_last_hour_performance()   - Last hour qty/retailers (DWH)

Note: Market prices use MODULE_1_INPUT data
Retailer Selection Queries defined ✓
  - get_churned_dropped_retailers()
  - get_category_not_product_retailers()
  - get_out_of_cycle_retailers()
  - get_view_no_orders_retailers()
  - get_excluded_retailers()
  - get_retailers_with_quantity_discount()
  - get_retailer_main_warehouse()


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


Market Data Module loaded at 2026-02-06 23:06:17 Cairo time
Snowflake timezone: America/Los_Angeles
All queries defined ✓
Helper functions defined ✓
get_market_data() function defined ✓
get_margin_tiers() function defined ✓

MARKET DATA MODULE READY

Available functions (NO INPUT REQUIRED):
  - get_market_data()   : Fetch and process all market prices
  - get_margin_tiers()  : Fetch and calculate margin tiers

Usage:
  %run market_data_module.ipynb
  df_market = get_market_data()
  df_tiers = get_margin_tiers()
Module 3: Periodic Actions
Run Time (Cairo): 2026-02-06 23:06:18
Current Hour (Cairo): 23
Input: MATERIALIZED_VIEWS.Pricing_data_extraction (today's data)


In [3]:
# =============================================================================
# LOAD PREVIOUS ACTIONS (Track price reductions per day)
# Now loads from Snowflake instead of local Excel files
# =============================================================================

def load_previous_actions():
    """Load previous Module 3 outputs from today (from Snowflake) to track price reductions."""
    try:
        # Query today's previous actions from Snowflake
        query = f"""
        SELECT * FROM {PREVIOUS_OUTPUT_TABLE}
        WHERE DATE(created_at) = '{TODAY}'
        ORDER BY created_at
        """
        df = query_snowflake(query)
        
        if len(df) == 0:
            print("No previous Module 3 outputs found for today. This is the first run.")
            return pd.DataFrame()
        
        print(f"Loaded {len(df)} previous action records from Snowflake")
        return df
    except Exception as e:
        print(f"Error loading previous actions from Snowflake: {e}")
        print("This may be the first run or table doesn't exist yet.")
        return pd.DataFrame()

def count_price_reductions_today(product_id, warehouse_id, previous_df):
    """Count how many price reductions this SKU has had today."""
    if previous_df.empty:
        return 0
    
    mask = (
        (previous_df['product_id'] == product_id) & 
        (previous_df['warehouse_id'] == warehouse_id) &
        (previous_df['price_action'].str.contains('decrease', na=False))
    )
    return mask.sum()
def count_price_increase_today(product_id, warehouse_id, previous_df):
    """Count how many price increase this SKU has had today."""
    if previous_df.empty:
        return 0
    
    mask = (
        (previous_df['product_id'] == product_id) & 
        (previous_df['warehouse_id'] == warehouse_id) &
        (previous_df['price_action'].str.contains('increase', na=False))
    )
    return mask.sum()
    

print("Loading previous actions from today...")
df_previous_actions = load_previous_actions()
print(f"Previous actions loaded: {len(df_previous_actions)} records")


Loading previous actions from today...


Loaded 56294 previous action records from Snowflake
Previous actions loaded: 56294 records


In [4]:
try:
    prev_inc = (
        df_previous_actions.assign(
            inc_flag=df_previous_actions['price_action'].str.contains('increase', case=False, na=False)
        )
        .groupby(['product_id', 'warehouse_id'])['inc_flag']
        .sum()
        .reset_index(name='increase_count')
    )
except:
    prev_inc = pd.DataFrame(columns=['product_id', 'warehouse_id','increase_count'])
try:    
    prev_red = (
    df_previous_actions.assign(
        red_flag=df_previous_actions['price_action'].str.contains('decrease', case=False, na=False)
    )
    .groupby(['product_id', 'warehouse_id'])['red_flag']
    .sum()
    .reset_index(name='reduced_count') 
    )
except:
    prev_red = pd.DataFrame(columns=['product_id', 'warehouse_id','reduced_count'])

In [5]:
# =============================================================================
# SNOWFLAKE CONNECTION
# =============================================================================
# query_snowflake() and TIMEZONE are provided by queries_module.ipynb
# (which also initializes Snowflake credentials from setup_environment_2)
print(f"Snowflake connection ready")
print(f"Timezone: {TIMEZONE}")


Snowflake connection ready
Timezone: America/Los_Angeles


In [6]:
# =============================================================================
# QUERY 1: TODAY'S UTH PERFORMANCE
# =============================================================================
UTH_LIVE_QUERY = f'''
WITH params AS (
    SELECT
        CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE AS today,
        HOUR(CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())) AS current_hour
),

-- Map dynamic tags to warehouse IDs using name matching
qd_det AS (
    SELECT DISTINCT 
        dt.id AS tag_id, 
        dt.name AS tag_name,
        REPLACE(w.name, ' ', '') AS warehouse_name,
        w.id AS warehouse_id,
        warehouse_name ILIKE '%' || CASE 
            WHEN SPLIT_PART(tag_name, '_', 1) = 'El' THEN SPLIT_PART(tag_name, '_', 2) 
            ELSE SPLIT_PART(tag_name, '_', 1) 
        END || '%' AS contains_flag
    FROM dynamic_tags dt
    JOIN dynamic_taggables dta ON dt.id = dta.dynamic_tag_id 
    CROSS JOIN warehouses w 
    WHERE dt.id > 3000
        AND dt.name LIKE '%QD_rets%'
        AND w.id IN (1, 236, 337, 8, 339, 170, 501, 401, 703, 632, 797, 962)
        AND contains_flag = 'true'
),

-- Get current active QD configurations
qd_config AS (
    SELECT * 
    FROM (
        SELECT 
            product_id,
            start_at,
            end_at,
            packing_unit_id,
            id AS qd_id,
            qd.warehouse_id,
            MAX(CASE WHEN tier = 1 THEN quantity END) AS tier_1_qty,
            MAX(CASE WHEN tier = 1 THEN discount_percentage END) AS tier_1_discount_pct,
            MAX(CASE WHEN tier = 2 THEN quantity END) AS tier_2_qty,
            MAX(CASE WHEN tier = 2 THEN discount_percentage END) AS tier_2_discount_pct,
            MAX(CASE WHEN tier = 3 THEN quantity END) AS tier_3_qty,
            MAX(CASE WHEN tier = 3 THEN discount_percentage END) AS tier_3_discount_pct
        FROM (
            SELECT 
                qd.id,
                qdv.product_id,
                qdv.packing_unit_id,
                qdv.quantity,
                qdv.discount_percentage,
                qd.dynamic_tag_id,
                qd.start_at,
                qd.end_at,
                ROW_NUMBER() OVER (
                    PARTITION BY qdv.product_id, qdv.packing_unit_id, qd.id 
                    ORDER BY qdv.quantity
                ) AS tier
            FROM quantity_discounts qd 
            JOIN quantity_discount_values qdv ON qdv.quantity_discount_id = qd.id
            WHERE active = 'true'
        ) qd_tiers
        JOIN qd_det qd ON qd.tag_id = qd_tiers.dynamic_tag_id
        GROUP BY ALL
    )
    QUALIFY ROW_NUMBER() OVER (PARTITION BY product_id, packing_unit_id, warehouse_id ORDER BY start_at DESC) = 1
),

-- Today's sales up-till-hour with discount breakdown
today_uth_sales AS (
    SELECT 
        pso.warehouse_id,
        pso.product_id,
        so.retailer_id,
        pso.packing_unit_id,
        pso.purchased_item_count AS qty,
        pso.total_price AS nmv,
        pso.ITEM_DISCOUNT_VALUE AS sku_discount_per_unit,
        pso.ITEM_QUANTITY_DISCOUNT_VALUE AS qty_discount_per_unit,
        qd.tier_1_qty,
        qd.tier_2_qty,
        qd.tier_3_qty,
        -- Determine tier used
        CASE 
            WHEN pso.ITEM_QUANTITY_DISCOUNT_VALUE = 0 OR qd.tier_1_qty IS NULL THEN 'Base'
            WHEN qd.tier_3_qty IS NOT NULL AND pso.purchased_item_count >= qd.tier_3_qty THEN 'Tier 3'
            WHEN qd.tier_2_qty IS NOT NULL AND pso.purchased_item_count >= qd.tier_2_qty THEN 'Tier 2'
            WHEN qd.tier_1_qty IS NOT NULL AND pso.purchased_item_count >= qd.tier_1_qty THEN 'Tier 1'
            ELSE 'Base'
        END AS tier_used
    FROM product_sales_order pso
    JOIN sales_orders so ON so.id = pso.sales_order_id
    LEFT JOIN qd_config qd 
        ON qd.product_id = pso.product_id 
        AND qd.packing_unit_id = pso.packing_unit_id
        AND qd.warehouse_id = so.warehouse_id
    CROSS JOIN params p
    WHERE so.created_at::DATE = p.today
        AND HOUR(so.created_at) < p.current_hour
        AND so.sales_order_status_id NOT IN (7, 12)
        AND so.channel IN ('telesales', 'retailer')
        AND pso.purchased_item_count <> 0
)

SELECT 
    warehouse_id,
    product_id,
    SUM(qty) AS uth_qty,
    SUM(nmv) AS uth_nmv,
    COUNT(DISTINCT retailer_id) AS uth_retailers,
    -- SKU discount NMV and contribution
    SUM(CASE WHEN sku_discount_per_unit > 0 THEN nmv ELSE 0 END) AS sku_discount_nmv_uth,
    ROUND(SUM(CASE WHEN sku_discount_per_unit > 0 THEN nmv ELSE 0 END) * 100.0 / NULLIF(SUM(nmv), 0), 2) AS sku_disc_cntrb_uth,
    -- Quantity discount NMV and contribution
    SUM(CASE WHEN qty_discount_per_unit > 0 THEN nmv ELSE 0 END) AS qty_discount_nmv_uth,
    ROUND(SUM(CASE WHEN qty_discount_per_unit > 0 THEN nmv ELSE 0 END) * 100.0 / NULLIF(SUM(nmv), 0), 2) AS qty_disc_cntrb_uth,
    -- Tier-level NMV
    SUM(CASE WHEN tier_used = 'Tier 1' THEN nmv ELSE 0 END) AS t1_nmv_uth,
    SUM(CASE WHEN tier_used = 'Tier 2' THEN nmv ELSE 0 END) AS t2_nmv_uth,
    SUM(CASE WHEN tier_used = 'Tier 3' THEN nmv ELSE 0 END) AS t3_nmv_uth,
    -- Tier-level contributions
    ROUND(SUM(CASE WHEN tier_used = 'Tier 1' THEN nmv ELSE 0 END) * 100.0 / NULLIF(SUM(nmv), 0), 2) AS t1_cntrb_uth,
    ROUND(SUM(CASE WHEN tier_used = 'Tier 2' THEN nmv ELSE 0 END) * 100.0 / NULLIF(SUM(nmv), 0), 2) AS t2_cntrb_uth,
    ROUND(SUM(CASE WHEN tier_used = 'Tier 3' THEN nmv ELSE 0 END) * 100.0 / NULLIF(SUM(nmv), 0), 2) AS t3_cntrb_uth
FROM today_uth_sales
GROUP BY warehouse_id, product_id
HAVING SUM(nmv) > 0
'''

print("Loading today's UTH performance with discount contributions...")
df_uth_today = query_snowflake(UTH_LIVE_QUERY)
print(f"Loaded {len(df_uth_today)} UTH records")


Loading today's UTH performance with discount contributions...


Loaded 10247 UTH records


In [7]:
# =============================================================================
# QUERY 2: HISTORICAL HOURLY DISTRIBUTION (Last 4 Months) - By Category & Warehouse
# =============================================================================
# Uses get_hourly_distribution() from queries_module

df_hourly_dist = get_hourly_distribution()

# Rename column for backwards compatibility with rest of Module 3
df_hourly_dist['avg_uth_pct'] = df_hourly_dist['avg_uth_pct_qty']
print(f"Using avg_uth_pct_qty as avg_uth_pct for Module 3 compatibility")


Fetching hourly distribution from Snowflake...


  Loaded 771 hourly distribution records
Using avg_uth_pct_qty as avg_uth_pct for Module 3 compatibility


In [8]:
# =============================================================================
# QUERY 3 & 4: ACTIVE DISCOUNTS
# =============================================================================

# SKU Discounts query (from data_extraction.ipynb)
ACTIVE_SKU_DISCOUNTS_QUERY = f'''
WITH active_sku_discount AS ( 
    SELECT 
        x.id AS sku_discount_id,
        retailer_id,
        product_id,
        packing_unit_id,
        DISCOUNT_PERCENTAGE,
        start_at,
        end_at 
    FROM (
        SELECT 
            sd.*,
            f.value::INT AS retailer_id 
        FROM SKU_DISCOUNTS sd,
        LATERAL FLATTEN(
            input => SPLIT(
                REPLACE(REPLACE(REPLACE(sd.retailer_ids, '{{', ''), '}}', ''), '"', ''), 
                ','
            )
        ) f
        WHERE start_at::DATE <= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
        and end_at::DATE >= CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP())::DATE
            AND active = 'true'
    ) x 
    JOIN SKU_DISCOUNT_VALUES sdv ON x.id = sdv.sku_discount_id
    WHERE name_en = 'Special Discounts'
    QUALIFY MAX(start_at) OVER (PARTITION BY retailer_id, product_id, packing_unit_id) = start_at 
)

SELECT 
    product_id, 
    warehouse_id,
    AVG(DISCOUNT_PERCENTAGE) AS active_sku_disc_pct,
    1 AS has_active_sku_discount
FROM (
    SELECT 
        asd.*,
        warehouse_id 
    FROM active_sku_discount asd 
    JOIN materialized_views.retailer_polygon rp ON rp.retailer_id = asd.retailer_id
    JOIN WAREHOUSE_DISPATCHING_RULES wdr ON wdr.product_id = asd.product_id
    JOIN DISPATCHING_POLYGONS dp ON dp.id = wdr.DISPATCHING_POLYGON_ID AND dp.district_id = rp.district_id
)
GROUP BY ALL
'''

# Active QD Query - Reuses the same CTE structure from UTH_LIVE_QUERY
ACTIVE_QD_QUERY = f'''
WITH qd_det AS (
    SELECT DISTINCT 
        dt.id AS tag_id, 
        dt.name AS tag_name,
        REPLACE(w.name, ' ', '') AS warehouse_name,
        w.id AS warehouse_id,
        warehouse_name ILIKE '%' || CASE 
            WHEN SPLIT_PART(tag_name, '_', 1) = 'El' THEN SPLIT_PART(tag_name, '_', 2) 
            ELSE SPLIT_PART(tag_name, '_', 1) 
        END || '%' AS contains_flag
    FROM dynamic_tags dt
    JOIN dynamic_taggables dta ON dt.id = dta.dynamic_tag_id 
    CROSS JOIN warehouses w 
    WHERE dt.id > 3000
        AND dt.name LIKE '%QD_rets%'
        AND w.id IN (1, 236, 337, 8, 339, 170, 501, 401, 703, 632, 797, 962)
        AND contains_flag = 'true'
),

qd_config AS (
    SELECT * 
    FROM (
        SELECT 
            product_id,
            packing_unit_id,
            qd.warehouse_id,
            MAX(CASE WHEN tier = 1 THEN quantity END) AS qd_tier_1_qty,
            MAX(CASE WHEN tier = 1 THEN discount_percentage END) AS qd_tier_1_disc_pct,
            MAX(CASE WHEN tier = 2 THEN quantity END) AS qd_tier_2_qty,
            MAX(CASE WHEN tier = 2 THEN discount_percentage END) AS qd_tier_2_disc_pct,
            MAX(CASE WHEN tier = 3 THEN quantity END) AS qd_tier_3_qty,
            MAX(CASE WHEN tier = 3 THEN discount_percentage END) AS qd_tier_3_disc_pct
        FROM (
            SELECT 
                qd.id,
                qdv.product_id,
                qdv.packing_unit_id,
                qdv.quantity,
                qdv.discount_percentage,
                qd.dynamic_tag_id,
                qd.start_at,
                ROW_NUMBER() OVER (
                    PARTITION BY qdv.product_id, qdv.packing_unit_id, qd.id 
                    ORDER BY qdv.quantity
                ) AS tier
            FROM quantity_discounts qd 
            JOIN quantity_discount_values qdv ON qdv.quantity_discount_id = qd.id
            WHERE  active = TRUE
        ) qd_tiers
        JOIN qd_det qd ON qd.tag_id = qd_tiers.dynamic_tag_id
        GROUP BY ALL
    )
    QUALIFY ROW_NUMBER() OVER (PARTITION BY product_id, packing_unit_id, warehouse_id ORDER BY qd_tier_1_qty DESC) = 1
)

SELECT 
    product_id,
    warehouse_id,
    qd_tier_1_qty,
    qd_tier_1_disc_pct,
    qd_tier_2_qty,
    qd_tier_2_disc_pct,
    qd_tier_3_qty,
    qd_tier_3_disc_pct,
    1 AS has_active_qd
FROM qd_config
'''

print("Loading active SKU discounts...")
df_active_sku_disc = query_snowflake(ACTIVE_SKU_DISCOUNTS_QUERY)
print(f"Loaded {len(df_active_sku_disc)} active SKU discount records")

print("Loading active Quantity discounts...")
df_active_qd = query_snowflake(ACTIVE_QD_QUERY)
print(f"Loaded {len(df_active_qd)} active QD records")


Loading active SKU discounts...


Loaded 5434 active SKU discount records
Loading active Quantity discounts...


Loaded 224 active QD records


In [9]:
# =============================================================================
# LOAD DATA FROM SNOWFLAKE (Instead of Excel file)
# =============================================================================
print("Loading data from Snowflake...")

# Query to get today's data from Pricing_data_extraction
LOAD_QUERY = f"""
SELECT * FROM {INPUT_TABLE}
WHERE created_at = '{datetime.now(CAIRO_TZ).date()}'
"""

df = query_snowflake(LOAD_QUERY)
print(f"Loaded {len(df)} records from Snowflake")

# Refresh live data using queries_module
print("\nRefreshing live data...")

# Refresh stocks
df_fresh_stocks = get_current_stocks()
df = df.drop(columns=['stocks'], errors='ignore')
df = df.merge(df_fresh_stocks, on=['warehouse_id', 'product_id'], how='left')
df['stocks'] = df['stocks'].fillna(0)

# Refresh current prices
df_fresh_prices = get_current_prices()
df = df.drop(columns=['current_price'], errors='ignore')
df = df.merge(df_fresh_prices[['cohort_id', 'product_id', 'current_price']], 
              on=['cohort_id', 'product_id'], how='left')

# Refresh WAC
df_fresh_wac = get_current_wac()
df = df.drop(columns=['wac_p'], errors='ignore')
df = df.merge(df_fresh_wac, on='product_id', how='left')

# Refresh cart rules
df_fresh_cart = get_current_cart_rules()
df = df.drop(columns=['current_cart_rule'], errors='ignore')
df = df.merge(df_fresh_cart, on=['cohort_id', 'product_id'], how='left')

print(f"Live data refreshed: stocks, prices, WAC, cart rules")

# Refresh market prices and margin tiers using new standalone functions
print("\nRefreshing market prices and margin tiers...")

# Get fresh market data (no input required)
df_fresh_market = get_market_data()
print(f"  Fetched {len(df_fresh_market)} market data records")

# Get fresh margin tiers (no input required)
df_fresh_tiers = get_margin_tiers()
print(f"  Fetched {len(df_fresh_tiers)} margin tier records")

# Drop old market columns and merge fresh data
market_cols_to_drop = [
    'below_market', 'market_min', 'market_25', 'market_50', 
    'market_75', 'market_max', 'above_market',
    'minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum',
    'ben_soliman_price', 'final_min_price', 'final_max_price', 'final_mod_price',
    'final_true_min', 'final_true_max', 'min_scrapped', 'scrapped25', 
    'scrapped50', 'scrapped75', 'max_scrapped'
]
df = df.drop(columns=[c for c in market_cols_to_drop if c in df.columns], errors='ignore')

# Merge fresh market data
df = df.merge(
    df_fresh_market, 
    on=['cohort_id', 'product_id','region'], 
    how='left'
)

# Drop old margin tier columns and merge fresh data
margin_tier_cols_to_drop = [
    'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3',
    'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2',
    'optimal_bm', 'min_boundary', 'max_boundary', 'median_bm',
    'effective_min_margin', 'margin_step'
]
df = df.drop(columns=[c for c in margin_tier_cols_to_drop if c in df.columns], errors='ignore')

# Merge fresh margin tiers
df = df.merge(
    df_fresh_tiers, 
    on=['cohort_id', 'product_id','region'], 
    how='left'
)

print(f"Market data refreshed")

# Merge UTH today data - drop old columns first
uth_cols = ['uth_qty', 'uth_nmv', 'uth_retailers', 'sku_discount_nmv_uth', 'sku_disc_cntrb_uth',
            'qty_discount_nmv_uth', 'qty_disc_cntrb_uth', 't1_nmv_uth', 't2_nmv_uth', 't3_nmv_uth',
            't1_cntrb_uth', 't2_cntrb_uth', 't3_cntrb_uth']
df = df.drop(columns=[c for c in uth_cols if c in df.columns], errors='ignore')

if len(df_uth_today) > 0:
    df = df.merge(df_uth_today, on=['warehouse_id', 'product_id'], how='left')
else:
    for col in uth_cols:
        df[col] = 0

# Merge hourly distribution - drop old column first (now by warehouse_id + cat)
df = df.drop(columns=['avg_uth_pct'], errors='ignore')
if len(df_hourly_dist) > 0:
    df = df.merge(df_hourly_dist, on=['warehouse_id', 'cat'], how='left')
else:
    df['avg_uth_pct'] = 0.5  # Default 50%

# Merge active SKU discounts - drop old columns first
sku_disc_cols = ['has_active_sku_discount', 'active_sku_disc_pct', 'active_sku_discount_value']
df = df.drop(columns=[c for c in sku_disc_cols if c in df.columns], errors='ignore')

if len(df_active_sku_disc) > 0:
    df = df.merge(df_active_sku_disc, on=['warehouse_id', 'product_id'], how='left')
else:
    df['has_active_sku_discount'] = 0
    df['active_sku_disc_pct'] = 0

# Merge active QD - drop old columns first
qd_cols = ['has_active_qd', 'qd_tier_1_qty', 'qd_tier_1_disc_pct', 
           'qd_tier_2_qty', 'qd_tier_2_disc_pct', 'qd_tier_3_qty', 'qd_tier_3_disc_pct']
df = df.drop(columns=[c for c in qd_cols if c in df.columns], errors='ignore')

if len(df_active_qd) > 0:
    df = df.merge(df_active_qd, on=['warehouse_id', 'product_id'], how='left')
else:
    df['has_active_qd'] = 0
    df['qd_tier_1_qty'] = 0
    df['qd_tier_1_disc_pct'] = 0
    df['qd_tier_2_qty'] = 0
    df['qd_tier_2_disc_pct'] = 0
    df['qd_tier_3_qty'] = 0
    df['qd_tier_3_disc_pct'] = 0

# Fill NaN
df['uth_qty'] = df['uth_qty'].fillna(0)
df['uth_retailers'] = df['uth_retailers'].fillna(0)
df['avg_uth_pct'] = df['avg_uth_pct'].fillna(0.5)
df['has_active_sku_discount'] = df['has_active_sku_discount'].fillna(0)
df['active_sku_discount_value'] = df.get('active_sku_discount_value', pd.Series([0]*len(df))).fillna(0)
df['has_active_qd'] = df['has_active_qd'].fillna(0)
df['qd_tier_1_qty'] = df['qd_tier_1_qty'].fillna(0)
df['qd_tier_1_disc_pct'] = df['qd_tier_1_disc_pct'].fillna(0)
df['qd_tier_2_qty'] = df['qd_tier_2_qty'].fillna(0)
df['qd_tier_2_disc_pct'] = df['qd_tier_2_disc_pct'].fillna(0)
df['qd_tier_3_qty'] = df['qd_tier_3_qty'].fillna(0)
df['qd_tier_3_disc_pct'] = df['qd_tier_3_disc_pct'].fillna(0)

# =============================================================================
# TURNOVER-BASED DOH: Calculate responsive_doh and min_induced_price (vectorized)
# =============================================================================
# responsive_doh = stocks / yesterday_qty (yesterday_qty comes from INPUT_TABLE)
df['yesterday_qty'] = pd.to_numeric(df.get('yesterday_qty', 0), errors='coerce').fillna(0)
df['responsive_doh'] = np.where(
    df['yesterday_qty'] > 0,
    df['stocks'] / df['yesterday_qty'],
    999  # No sales yesterday = infinite DOH
)

# min_induced_price = wac_p * (0.9 + target_margin * 0.5)
# This is the floor price for induced pricing when DOH > 30
df['target_margin'] = pd.to_numeric(df.get('target_margin', 0), errors='coerce').fillna(0)
df['min_induced_price'] = df['wac_p'] * (0.9 + df['target_margin'] * 0.5)

print(f"Data merged. Total records: {len(df)}")
print(f"  SKUs with active SKU discount: {(df['has_active_sku_discount'] == 1).sum()}")
print(f"  SKUs with active QD: {(df['has_active_qd'] == 1).sum()}")
print(f"  SKUs with high DOH (>30): {(df['responsive_doh'] > 30).sum()}")


Loading data from Snowflake...


Loaded 28147 records from Snowflake

Refreshing live data...
Fetching current stocks...


  Loaded 1865053 records


Fetching current prices...


  Loaded 116402 records
Fetching current WAC...


...

/tmp/ipykernel_25499/3245917641.py:139: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


    Records after group processing: 24888

Step 5: Adding WAC and margin data...
    Records with WAC: 24588

Step 6: Filtering by price coverage...
    Records after price coverage filter: 13547

Step 7: Calculating price percentiles...


    Records after price analysis: 12780

Step 8: Converting prices to margins...

MARKET DATA COMPLETE
Total records: 12780
  - With marketplace prices: 12053
  - With scrapped prices: 6231
  - With Ben Soliman prices: 8364
  Fetched 12780 market data records

FETCHING MARGIN TIERS
Timestamp: 2026-02-06 23:07:28 Cairo time

Step 1: Fetching margin boundaries from PRODUCT_STATISTICS...


    Loaded 18028 records

Step 2: Adding cohort IDs...
    Records with cohorts: 24868

Step 3: Calculating margin tiers...

MARGIN TIERS COMPLETE
Total records: 24868

Margin Tier Structure:
  margin_tier_below:   effective_min - step (1 below)
  margin_tier_1:       effective_min_margin
  margin_tier_2:       effective_min + 1*step
  margin_tier_3:       effective_min + 2*step
  margin_tier_4:       effective_min + 3*step
  margin_tier_5:       max_boundary
  margin_tier_above_1: max_boundary + 1*step
  margin_tier_above_2: max_boundary + 2*step
  Fetched 24868 margin tier records
Market data refreshed


Data merged. Total records: 28155
  SKUs with active SKU discount: 5395
  SKUs with active QD: 224
  SKUs with high DOH (>30): 18532


In [10]:
# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def calculate_margin(price, wac):
    if pd.isna(price) or pd.isna(wac) or price == 0:
        return None
    return (price - wac) / price

def get_market_tiers(row):
    """Get sorted list of market price tiers."""
    tiers = []
    for col in ['minimum', 'percentile_25', 'percentile_50', 'percentile_75', 'maximum']:
        val = row.get(col)
        if pd.notna(val) and val > 0:
            tiers.append(val)
    return sorted(set(tiers))

def get_margin_tiers(row):
    """Get sorted list of margin-based price tiers (converted to prices)."""
    tiers = []
    wac = row.get('wac_p', 0)
    if wac <= 0:
        return tiers
    
    for tier_col in ['margin_tier_below','margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
                     'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2']:
        margin = row.get(tier_col)
        if pd.notna(margin) and 0 < margin < 1:
            price = wac / (1 - margin)
            tiers.append(round(price, 2))
    return sorted(set(tiers))

def find_next_price_above(current_price, row):
    """
    Find the first price tier ABOVE current_price by at least MIN_PRICE_CHANGE_EGP.
    Market first, then margin. Skips tiers less than 0.25 EGP above.
    """
    current_price = float(current_price) if current_price else 0
    if pd.isna(current_price) or current_price <= 0:
        return current_price
    
    for tier in get_market_tiers(row):
        if tier > current_price + MIN_PRICE_CHANGE_EGP:
            return round(tier, 2)
    
    for tier in get_margin_tiers(row):
        if tier > current_price + MIN_PRICE_CHANGE_EGP:
            return round(tier, 2)
    
    return current_price

def find_next_price_below(current_price, row):
    """
    Find the first price tier BELOW current_price by at least MIN_PRICE_CHANGE_EGP.
    Market first, then margin. Skips tiers less than 0.25 EGP below.
    """
    current_price = float(current_price) if current_price else 0
    if pd.isna(current_price) or current_price <= 0:
        return current_price
    
    for tier in reversed(get_market_tiers(row)):
        if tier < current_price - MIN_PRICE_CHANGE_EGP:
            return round(tier, 2)
    
    for tier in reversed(get_margin_tiers(row)):
        if tier < current_price - MIN_PRICE_CHANGE_EGP:
            return round(tier, 2)
    
    return current_price

def find_price_n_steps_below(current_price, n_steps, row):
    """Find price N steps below current (iteratively find next tier below)."""
    price = current_price
    for _ in range(n_steps):
        next_price = find_next_price_below(price, row)
        if next_price >= price:  # No tier below found
            break
        price = next_price
    return price

def is_cart_too_open(row):
    """Check if cart rule is too open: > normal_refill + 10*std"""
    normal_refill = float(row.get('normal_refill', 5) or 5)
    stddev = float(row.get('refill_stddev', 2) or 2)
    current_cart = float(row.get('cart_rule', normal_refill) or normal_refill)
    threshold = normal_refill + (10 * stddev)
    return current_cart > threshold

def adjust_cart_rule(current_cart, direction, row):
    """Adjust cart rule by 20% up or down."""
    normal_refill = float(row.get('normal_refill', 5) or 5)
    stddev = float(row.get('refill_stddev', 2) or 2)
    current_cart = float(current_cart or 5)
    
    if direction == 'increase':
        new_cart = current_cart * (1 + CART_INCREASE_PCT)
        new_cart = min(new_cart, MAX_CART_RULE)
    else:  # decrease
        # Formula: max(0.8 * cart, normal_refill + 3*std)
        new_cart = current_cart * (1 - CART_DECREASE_PCT)
        min_floor = normal_refill + (3 * stddev)
        new_cart = max(new_cart, min_floor, MIN_CART_RULE)
    
    return int(new_cart)

def get_highest_qd_tier_contribution(row):
    """Find which QD tier has highest contribution."""
    t1 = row.get('yesterday_t1_cntrb', 0) or 0
    t2 = row.get('yesterday_t2_cntrb', 0) or 0
    t3 = row.get('yesterday_t3_cntrb', 0) or 0
    
    if t1 >= t2 and t1 >= t3 and t1 > 0:
        return 'T1', t1
    elif t2 >= t1 and t2 >= t3 and t2 > 0:
        return 'T2', t2
    elif t3 > 0:
        return 'T3', t3
    return None, 0

print("Helper functions loaded.")


Helper functions loaded.


In [11]:
# =============================================================================
# HELPER: Calculate margin step from existing tier prices
# =============================================================================
def calculate_margin_step(row):
    """
    Calculate the margin step size from existing margin tiers.
    Used to induce prices below available tiers when DOH > 30.
    
    Returns:
        Average step size between consecutive tiers, or 0.015 (1.5%) as default
    """
    tier_cols = ['margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
                 'margin_tier_4', 'margin_tier_5']
    tiers = [row.get(col) for col in tier_cols]
    valid_tiers = [t for t in tiers if pd.notna(t) and t is not None]
    
    if len(valid_tiers) >= 2:
        # Calculate steps between consecutive tiers
        steps = [abs(valid_tiers[i+1] - valid_tiers[i]) for i in range(len(valid_tiers)-1)]
        return np.mean(steps) if steps else 0.01
    
    # Fallback: use market margins if available
    market_cols = ['market_min', 'market_25', 'market_50', 'market_75', 'market_max']
    markets = [row.get(col) for col in market_cols]
    valid_markets = [m for m in markets if pd.notna(m) and m is not None]
    
    if len(valid_markets) >= 2:
        steps = [abs(valid_markets[i+1] - valid_markets[i]) for i in range(len(valid_markets)-1)]
        return np.mean(steps) if steps else 0.01
    
    return 0.01 # Default 1% step

def calculate_induced_price(row, current_price):
    """
    Calculate induced price by reducing margin by one step.
    Used for Zero Demand and High DOH scenarios.
    
    Returns:
        Induced price if valid and lower than current, else None
    """
    wac_p = float(row.get('wac_p', 0) or 0)
    if wac_p <= 0 or current_price <= 0:
        return None
    
    current_margin = (current_price - wac_p) / current_price
    margin_step = calculate_margin_step(row)
    new_margin = current_margin - margin_step
    
    if new_margin >= 1:
        return None
    
    induced_price = wac_p / (1 - new_margin)
    induced_price = round(induced_price * 4) / 4  # Round to 0.25
    
    # Apply floors: min_induced_price and commercial_min_price
    min_induced = float(row.get('min_induced_price', 0) or 0)
    commercial_min = float(row.get('commercial_min_price', 0) or 0)
    floor_price = max(min_induced, commercial_min) if commercial_min > 0 else min_induced
    
    if induced_price < floor_price:
        return None  # Can't reduce further
    
    return induced_price if induced_price < current_price else None

# =============================================================================
# MAIN ENGINE: GENERATE PERIODIC ACTION
# =============================================================================

def generate_periodic_action(row, previous_df):
    """
    Generate periodic action based on UTH performance.
    
    Logic:
    - Zero Demand: 1 step below current + SKU discount
    - On Track: No action
    - Growing: Deactivate discounts or increase price, reduce cart if too open
    - Dropping: Based on qty_ratio vs retailer_ratio:
        - qty OK, retailers dropping: SKU discount (then price if already has)
        - qty dropping, retailers OK: QD (then price if already has)
        - both dropping: SKU discount (then price if already has)
    - Price reduction max 2x per day
    """
    product_id = row.get('product_id')
    warehouse_id = row.get('warehouse_id')
    
    result = {
        'product_id': product_id,
        'warehouse_id': warehouse_id,
        'cohort_id': row.get('cohort_id'),
        'sku': row.get('sku'),
        'brand': row.get('brand'),
        'cat': row.get('cat'),
        'stocks': row.get('stocks', 0),
        'current_price': row.get('current_price'),
        'wac_p': row.get('wac_p'),
        'uth_qty': row.get('uth_qty', 0),
        'uth_retailers': row.get('uth_retailers', 0),
        'p80_daily_240d': row.get('p80_daily_240d', 1),
        'p70_daily_retailers_240d': row.get('p70_daily_retailers_240d', 1),
        'avg_uth_pct': row.get('avg_uth_pct', 0.5),
        # Today's UTH discount contributions
        'sku_disc_cntrb_uth': row.get('sku_disc_cntrb_uth', 0) or 0,
        't1_cntrb_uth': row.get('t1_cntrb_uth', 0) or 0,
        't2_cntrb_uth': row.get('t2_cntrb_uth', 0) or 0,
        't3_cntrb_uth': row.get('t3_cntrb_uth', 0) or 0,
        'uth_status': None,
        'qty_ratio': None,
        'retailer_ratio': None,
        'new_price': None,
        'price_action': None,
        'current_cart_rule':row.get('current_cart_rule'),
        'new_cart_rule': None,
        'activate_sku_discount': False,  # True = SKU should have discount after this run
        'activate_qd': False,             # True = SKU should have QD after this run
        'keep_qd_tiers': None,            # List of QD tiers to keep (e.g., ['T1', 'T2'])
        # QD tier configuration (passed to qd_handler)
        'qd_tier_1_qty': row.get('qd_tier_1_qty', 0) or 0,
        'qd_tier_1_disc_pct': row.get('qd_tier_1_disc_pct', 0) or 0,
        'qd_tier_2_qty': row.get('qd_tier_2_qty', 0) or 0,
        'qd_tier_2_disc_pct': row.get('qd_tier_2_disc_pct', 0) or 0,
        'qd_tier_3_qty': row.get('qd_tier_3_qty', 0) or 0,
        'qd_tier_3_disc_pct': row.get('qd_tier_3_disc_pct', 0) or 0,
        'removed_discount': None,         # Which discount was removed (for Growing)
        'removed_discount_cntrb': 0,      # Contribution of removed discount
        'price_reductions_today': row.get('reduced_count', 0) or 0,
        'action_reason': None,
        # =====================================================================
        # ADDITIONAL COLUMNS FOR QD AND SKU DISCOUNT HANDLERS
        # =====================================================================
        # Pricing and margin data
        'target_margin': row.get('target_margin'),
        'min_boundary': row.get('min_boundary'),
        'doh': row.get('doh', 0),  # Days on hand - for SKU discount handler
        'mtd_qty': row.get('mtd_qty', 0),  # MTD quantity - for QD ranking
        # Active SKU discount info - for SKU discount handler
        'active_sku_disc_pct': row.get('active_sku_disc_pct', 0),
        'has_active_sku_discount': row.get('has_active_sku_discount', 0),
        'has_active_qd': row.get('has_active_qd', 0),
        # Market margins (converted to prices in handlers)
        'below_market': row.get('below_market'),
        'market_min': row.get('market_min'),
        'market_25': row.get('market_25'),
        'market_50': row.get('market_50'),
        'market_75': row.get('market_75'),
        'market_max': row.get('market_max'),
        'above_market': row.get('above_market'),
        # Margin tiers (converted to prices in handlers)
        'margin_tier_below': row.get('margin_tier_below'),
        'margin_tier_1': row.get('margin_tier_1'),
        'margin_tier_2': row.get('margin_tier_2'),
        'margin_tier_3': row.get('margin_tier_3'),
        'margin_tier_4': row.get('margin_tier_4'),
        'margin_tier_5': row.get('margin_tier_5'),
        'margin_tier_above_1': row.get('margin_tier_above_1'),
        'margin_tier_above_2': row.get('margin_tier_above_2'),
    }
    
    # Skip if OOS (price only in Module 2)
    if row.get('stocks', 0) <= 0:
        result['action_reason'] = 'OOS - skip (price only in Module 2)'
        return result
    
    # Skip if below minimum stock (stock < min selling unit qty)
    if row.get('below_min_stock_flag', 0) == 1:
        result['action_reason'] = 'Below min stock - skip (cannot sell)'
        return result
    
    # Count previous price reductions today
    price_reductions_today = row.get('reduced_count', 0)
    can_reduce_price = price_reductions_today < MAX_PRICE_REDUCTIONS_PER_DAY

    # Count previous price increase today
    price_increase_today = row.get('increase_count', 0)
    can_increase_price = price_increase_today < MAX_PRICE_REDUCTIONS_PER_DAY    
    
    # Calculate UTH benchmark: historical_pct * P80_qty
    # Convert to float to handle decimal.Decimal from Snowflake
    p80_qty = float(row.get('p80_daily_240d', 1) or 1)
    p70_retailers = float(row.get('p70_daily_retailers_240d', 1) or 1)
    avg_uth_pct = float(row.get('avg_uth_pct', 0.5) or 0.5)
    
    uth_qty_target = p80_qty * avg_uth_pct
    uth_retailer_target = p70_retailers * avg_uth_pct
    
    uth_qty = float(row.get('uth_qty', 0) or 0)
    uth_retailers = float(row.get('uth_retailers', 0) or 0)
    
    # Calculate UTH ratios
    qty_ratio = uth_qty / uth_qty_target if uth_qty_target > 0 else 0
    retailer_ratio = uth_retailers / uth_retailer_target if uth_retailer_target > 0 else 0
    
    result['uth_qty_target'] = round(uth_qty_target, 2)
    result['uth_retailer_target'] = round(uth_retailer_target, 2)
    result['qty_ratio'] = round(qty_ratio, 2)
    result['retailer_ratio'] = round(retailer_ratio, 2)
    
    current_price = float(row.get('current_price') or 0)
    current_cart = float(row.get('current_cart_rule', row.get('normal_refill', 10)) or 10)
    has_sku_disc = row.get('has_active_sku_discount', 0) == 1
    has_qd = row.get('has_active_qd', 0) == 1
    
    # Determine if qty/retailers are dropping (below threshold)
    qty_dropping = qty_ratio < UTH_DROPPING_THRESHOLD
    qty_ok = qty_ratio >= UTH_DROPPING_THRESHOLD
    retailer_dropping = retailer_ratio < UTH_DROPPING_THRESHOLD
    retailer_ok = retailer_ratio >= UTH_DROPPING_THRESHOLD
    
    # =========================================================================
    # CASE 1: Zero demand today
    # - If NO existing SKU discount: Add SKU discount ONLY (wait for next day)
    # - If HAS existing SKU discount: Keep discount + INDUCED price reduction
    # - Open cart if tight (both cases)
    # =========================================================================
    if uth_qty == 0:
        result['uth_status'] = 'Zero Demand'
        result['activate_sku_discount'] = True
        
        # Check if cart rule is tight (< normal_refill + 10*std) and increase if so
        normal_refill = float(row.get('normal_refill', 5) or 5)
        stddev = float(row.get('refill_stddev', 2) or 2)
        cart_threshold = normal_refill + (10 * stddev)
        
        if current_cart < cart_threshold:
            new_cart = min(cart_threshold, MAX_CART_RULE)
            new_cart = max(new_cart, MIN_CART_RULE)
            result['new_cart_rule'] = int(new_cart)
            cart_action = f' + open cart to {int(new_cart)}'
        else:
            cart_action = ''
        
        if not has_sku_disc:
            # First occurrence: Add SKU discount only - wait for next day
            result['price_action'] = 'add_sku_disc'
            result['action_reason'] = f'Zero demand - ADD SKU discount (wait for next day){cart_action}'
        else:
            # Second occurrence: Already has SKU discount but still 0 demand - reduce price
            induced_price = calculate_induced_price(row, current_price)
            if induced_price:
                result['new_price'] = induced_price
                result['price_action'] = 'zero_demand_induced_price'
                result['action_reason'] = f'Zero demand + existing discount - INDUCED price ({current_price:.2f} -> {induced_price:.2f}){cart_action}'
            else:
                result['price_action'] = 'keep_sku_disc'
                result['action_reason'] = f'Zero demand + existing discount - no lower price available{cart_action}'
        
        return result
    
    # =========================================================================
    # CASE 1.5: HIGH DOH (responsive_doh > 30) - Two-step approach
    # - If NO existing SKU discount: Add SKU discount ONLY (wait for next day)
    # - If HAS existing SKU discount and qty_ratio >= 0.9 ("grew"): Keep discount only
    # - If HAS existing SKU discount and qty_ratio < 0.9 ("didn't grow"): Keep discount + induced price
    # Only applies if inventory value (stocks * price) > 10,000 EGP
    # Skip SKUs that were out of stock yesterday (oos_yesterday = 1)
    # =========================================================================
    DOH_HIGH_TURNOVER_THRESHOLD = 30
    HIGH_INVENTORY_VALUE_THRESHOLD = 10000
    responsive_doh = float(row.get('responsive_doh', 999) or 999)
    stocks = float(row.get('stocks', 0) or 0)
    inventory_value = stocks * current_price
    oos_yesterday = int(row.get('oos_yesterday', 0) or 0)
    
    if responsive_doh > DOH_HIGH_TURNOVER_THRESHOLD and inventory_value > HIGH_INVENTORY_VALUE_THRESHOLD and oos_yesterday != 1 and row.get('doh', 0) >20 :
        result['uth_status'] = 'High DOH'
        result['activate_sku_discount'] = True
        
        if not has_sku_disc:
            # First occurrence: Add SKU discount only - wait for next day
            result['price_action'] = 'add_sku_disc_doh'
            result['action_reason'] = f'High DOH ({responsive_doh:.1f} days) - ADD SKU discount (wait for next day)'
            return result
        
        else:
            # Has existing SKU discount - check if "grew" (qty_ratio >= 0.9)
            if qty_ratio >= 0.9:
                # SKU "grew" - keep discount but don't reduce price
                result['price_action'] = 'keep_sku_disc'
                result['action_reason'] = f'High DOH ({responsive_doh:.1f} days) + grew (qty={qty_ratio:.2f}) - KEEP SKU discount only'
                return result
            else:
                # SKU "didn't grow" - keep discount + reduce price with induced logic
                if can_reduce_price:
                    induced_price = calculate_induced_price(row, current_price)
                    if induced_price:
                        result['new_price'] = induced_price
                        result['price_action'] = 'induced_doh_reduction'
                        result['action_reason'] = f'High DOH ({responsive_doh:.1f} days) + didn\'t grow (qty={qty_ratio:.2f}) - INDUCED price ({current_price:.2f} -> {induced_price:.2f})'
                        return result
                    else:
                        result['price_action'] = 'keep_sku_disc'
                        result['action_reason'] = f'High DOH ({responsive_doh:.1f} days) - no lower price available'
                        return result
                else:
                    result['price_action'] = 'keep_sku_disc'
                    result['action_reason'] = f'High DOH ({responsive_doh:.1f} days) - price reduction limit reached'
                    return result
    
    # =========================================================================
    # CASE 2: On Track (both qty and retailers ±10%)
    # If has existing discounts, keep them (they'll be deactivated otherwise)
    # =========================================================================
    if (UTH_DROPPING_THRESHOLD <= qty_ratio <= UTH_GROWING_THRESHOLD and
        UTH_DROPPING_THRESHOLD <= retailer_ratio <= UTH_GROWING_THRESHOLD):
        result['uth_status'] = 'On Track'
        result['price_action'] = 'hold'
        
        # Preserve existing discounts (all discounts are deactivated at start of each run)
        if has_sku_disc:
            result['activate_sku_discount'] = True
            result['action_reason'] = f'On Track (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f}) - keep existing SKU discount'
        elif has_qd:
            result['activate_qd'] = True
            result['action_reason'] = f'On Track (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f}) - keep existing QD'
        else:
            result['action_reason'] = f'On Track (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f}) - no action'
        
        return result
    
    # =========================================================================
    # CASE 2.5: Retailers Growing but Qty On Track
    # Action: Increase price 1 step (high retailer demand, normal qty = opportunity)
    # =========================================================================
    if (UTH_DROPPING_THRESHOLD <= qty_ratio <= UTH_GROWING_THRESHOLD and
        retailer_ratio > UTH_GROWING_THRESHOLD):
        result['uth_status'] = 'Retailers Growing'
        if can_increase_price:
            new_price = find_next_price_above(current_price, row)
        else:
            new_price = np.nan
        if new_price > current_price:
            result['new_price'] = new_price
            result['price_action'] = 'retailers_growing_increase'
            result['action_reason'] = f'Retailers growing (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f}) - increase price ({current_price:.2f} -> {new_price:.2f})'
        else:
            result['price_action'] = 'hold'
            result['action_reason'] = f'Retailers growing (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f}) - no tier above, hold'
        
        return result
    
    # =========================================================================
    # CASE 3: Growing (qty > 110%)
    # Find discount with HIGHEST contribution (from TODAY's UTH) and remove it
    # Keep (re-activate) the others
    # If no discounts -> increase price
    # =========================================================================
    if qty_ratio > UTH_GROWING_THRESHOLD:
        result['uth_status'] = 'Growing'
        
        # Get TODAY's UTH discount contributions (not yesterday's)
        sku_disc_cntrb = row.get('sku_disc_cntrb_uth', 0) or 0
        t1_cntrb = row.get('t1_cntrb_uth', 0) or 0
        t2_cntrb = row.get('t2_cntrb_uth', 0) or 0
        t3_cntrb = row.get('t3_cntrb_uth', 0) or 0
        
        # Build list of EXISTING discounts with their contributions
        # Note: We check if tiers EXIST (qty > 0), not just if they had sales today
        # A tier can exist but have 0 contribution if no orders used it yet today
        active_discounts = []
        
        # SKU discount: check if it exists (has_sku_disc from active discount query)
        if has_sku_disc:
            active_discounts.append(('sku_disc', sku_disc_cntrb))  # Include even if cntrb=0
        
        # QD tiers: check if each tier EXISTS (qty > 0 means the tier is configured)
        if has_qd:
            qd_t1_qty = row.get('qd_tier_1_qty', 0) or 0
            qd_t2_qty = row.get('qd_tier_2_qty', 0) or 0
            qd_t3_qty = row.get('qd_tier_3_qty', 0) or 0
            
            if qd_t1_qty > 0:  # Tier 1 exists
                active_discounts.append(('qd_t1', t1_cntrb))  # Include even if cntrb=0
            if qd_t2_qty > 0:  # Tier 2 exists
                active_discounts.append(('qd_t2', t2_cntrb))  # Include even if cntrb=0
            if qd_t3_qty > 0:  # Tier 3 exists
                active_discounts.append(('qd_t3', t3_cntrb))  # Include even if cntrb=0
        
        if active_discounts:
            # Sort by contribution descending - remove the highest
            active_discounts.sort(key=lambda x: x[1], reverse=True)
            highest_disc, highest_cntrb = active_discounts[0]
            remaining_discounts = [d[0] for d in active_discounts[1:]]
            
            # Determine what to keep (re-activate)
            keep_sku_disc = 'sku_disc' in remaining_discounts
            keep_qd_t1 = 'qd_t1' in remaining_discounts
            keep_qd_t2 = 'qd_t2' in remaining_discounts
            keep_qd_t3 = 'qd_t3' in remaining_discounts
            keep_any_qd = keep_qd_t1 or keep_qd_t2 or keep_qd_t3
            
            # Set activation flags
            if keep_sku_disc:
                result['activate_sku_discount'] = True
            
            if keep_any_qd:
                result['activate_qd'] = True
                result['keep_qd_tiers'] = [t for t in ['T1', 'T2', 'T3'] 
                                           if (t == 'T1' and keep_qd_t1) or 
                                              (t == 'T2' and keep_qd_t2) or 
                                              (t == 'T3' and keep_qd_t3)]
            
            result['removed_discount'] = highest_disc
            result['removed_discount_cntrb'] = highest_cntrb
            result['price_action'] = f'remove_{highest_disc}'
            result['action_reason'] = f'Growing (qty={qty_ratio:.2f}) - remove {highest_disc} (cntrb={highest_cntrb}%)'
            
            if remaining_discounts:
                result['action_reason'] += f', keep {remaining_discounts}'
        
        elif has_sku_disc or has_qd:
            # Has discounts but no contribution data - remove all
            result['price_action'] = 'remove_all_disc'
            result['action_reason'] = f'Growing (qty={qty_ratio:.2f}) - remove all discounts (no contribution data)'
        
        else:
            # No discounts
            result['price_action'] = 'no_discount_growing'
            result['action_reason'] = f'Growing (qty={qty_ratio:.2f}) - no discounts'
        
        # ALWAYS increase price 1 step (regardless of discounts)
        if can_increase_price:
            new_price = find_next_price_above(current_price, row)
            if pd.notna(new_price) and new_price > current_price:
                result['new_price'] = new_price
                result['action_reason'] += f' + increase price ({current_price:.2f} -> {new_price:.2f})'
            else:
                result['action_reason'] += ' + no tier above for price increase'
        else:
            result['action_reason'] += ' + price increase limit reached'
        
        # ALWAYS reduce cart rule (not just when too open)
        result['new_cart_rule'] = adjust_cart_rule(current_cart, 'decrease', row)
        result['action_reason'] += ' + reduce cart'
        
        return result
    
    # =========================================================================
    # CASE 4: Dropping - Different actions based on qty vs retailer ratios
    # =========================================================================
    result['uth_status'] = 'Dropping'
    
    def apply_price_reduction():
        """Helper to apply price reduction if allowed."""
        if not can_reduce_price:
            return None, f'Price reduction limit reached ({price_reductions_today}/{MAX_PRICE_REDUCTIONS_PER_DAY} today)'
        
        new_price = find_next_price_below(current_price, row)
        if new_price < current_price:
            commercial_min = float(row.get('commercial_min_price', row.get('minimum', 0)) or 0)
            if pd.notna(commercial_min) and commercial_min > 0:
                new_price = max(new_price, commercial_min)
            return new_price, f'decrease ({current_price:.2f} -> {new_price:.2f})'
        return None, 'no tier below'
    
    # CASE 4A: qty OK (≥90%) but retailers dropping (<90%)
    # Action: SKU discount (add new OR keep existing), then price if already has
    if qty_ok and retailer_dropping:
        # Always set activate_sku_discount = True (either adding new or keeping existing)
        result['activate_sku_discount'] = True
        
        if not has_sku_disc:
            # Adding new SKU discount
            result['price_action'] = 'add_sku_disc'
            result['action_reason'] = f'Retailers dropping (ret={retailer_ratio:.2f}, qty OK) - ADD new SKU discount'
        else:
            # Keeping existing SKU discount + reduce price
            new_price, reason = apply_price_reduction()
            if new_price:
                #result['new_price'] = new_price
                result['price_action'] = 'keep_sku_disc_and_decrease'
                result['action_reason'] = f'Retailers dropping - KEEP SKU disc + {reason}'
            else:
                result['price_action'] = 'keep_sku_disc'
                result['action_reason'] = f'Retailers dropping - KEEP SKU disc ({reason})'
    
    # CASE 4B: qty dropping (<90%) but retailers OK (≥90%)
    # Action: QD (add new OR keep existing), then price if already has
    elif qty_dropping and retailer_ok:
        # Always set activate_qd = True (either adding new or keeping existing)
        result['activate_qd'] = True
        
        if not has_qd:
            # Adding new QD
            result['price_action'] = 'add_qd'
            result['action_reason'] = f'Qty dropping (qty={qty_ratio:.2f}, ret OK) - ADD new QD'
        else:
            # Keeping existing QD + reduce price
            new_price, reason = apply_price_reduction()
            if new_price:
                result['new_price'] = new_price
                result['price_action'] = 'keep_qd_and_decrease'
                result['action_reason'] = f'Qty dropping - KEEP QD + {reason}'
            else:
                result['price_action'] = 'keep_qd'
                result['action_reason'] = f'Qty dropping - KEEP QD ({reason})'
    
    # CASE 4C: Both dropping (<90%)
    # Action: SKU discount (add new OR keep existing), then price if already has
    elif qty_dropping and retailer_dropping:
        # Always set activate_sku_discount = True (either adding new or keeping existing)
        result['activate_sku_discount'] = True
        
        if not has_sku_disc:
            # Adding new SKU discount
            result['price_action'] = 'add_sku_disc'
            result['action_reason'] = f'Both dropping (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f}) - ADD new SKU discount'
        else:
            # Keeping existing SKU discount + reduce price
            new_price, reason = apply_price_reduction()
            if new_price:
                result['new_price'] = new_price
                result['price_action'] = 'keep_sku_disc_and_decrease'
                result['action_reason'] = f'Both dropping - KEEP SKU disc + {reason}'
            else:
                result['price_action'] = 'keep_sku_disc'
                result['action_reason'] = f'Both dropping - KEEP SKU disc ({reason})'
    
    else:
        result['price_action'] = 'hold'
        result['action_reason'] = f'Unexpected state (qty={qty_ratio:.2f}, ret={retailer_ratio:.2f})'
    
    # Increase cart for dropping SKUs
    result['new_cart_rule'] = adjust_cart_rule(current_cart, 'increase', row)
    result['action_reason'] += ' + increase cart 20%'
    
    return result

print("Main engine function loaded.")


Main engine function loaded.


In [12]:
df = df.merge(prev_inc,on=['product_id','warehouse_id'],how='left')
df = df.merge(prev_red,on=['product_id','warehouse_id'],how='left')
df['increase_count'] = df['increase_count'].fillna(0)
df['reduced_count'] = df['reduced_count'].fillna(0)

In [13]:
# =============================================================================
# EXECUTE MODULE 3
# =============================================================================
print(f"Processing {len(df)} SKUs...")
print("="*60)

results = []
for idx, row in df.iterrows():
    result = generate_periodic_action(row, df_previous_actions)
    results.append(result)
    
    if (idx + 1) % 10000 == 0:
        print(f"Processed {idx + 1}/{len(df)} SKUs...")

df_results = pd.DataFrame(results)
print(f"\n✅ Processed {len(df_results)} SKUs")


Processing 28155 SKUs...


Processed 10000/28155 SKUs...


Processed 20000/28155 SKUs...



✅ Processed 28155 SKUs


In [14]:
df_results = df_results.drop_duplicates(subset=['product_id', 'warehouse_id'], keep='first')

In [15]:
# =============================================================================
# SUMMARY
# =============================================================================
print("="*60)
print("MODULE 3 SUMMARY")
print("="*60)

print(f"\nTotal SKUs: {len(df_results)}")

print(f"\nBy UTH Status:")
print(df_results['uth_status'].value_counts(dropna=False).to_string())

# Actions breakdown
price_changes = df_results[df_results['new_price'].notna()]
cart_changes = df_results[df_results['new_cart_rule'].notna()]
sku_disc_activate = df_results[df_results['activate_sku_discount'] == True]
qd_activate = df_results[df_results['activate_qd'] == True]
discounts_removed = df_results[df_results['removed_discount'].notna()]

print(f"\nActions:")
print(f"  Price changes: {len(price_changes)}")
print(f"  Cart rule changes: {len(cart_changes)}")
print(f"  SKU discounts to activate: {len(sku_disc_activate)}")
print(f"  QD to activate: {len(qd_activate)}")
print(f"  Discounts removed (Growing SKUs): {len(discounts_removed)}")


MODULE 3 SUMMARY

Total SKUs: 28147

By UTH Status:
uth_status
Zero Demand          9724
None                 9548
Dropping             7058
Growing              1063
High DOH              479
Retailers Growing     158
On Track              117

Actions:
  Price changes: 3096
  Cart rule changes: 11427
  SKU discounts to activate: 16700
  QD to activate: 620
  Discounts removed (Growing SKUs): 261


In [16]:
# =============================================================================
# EXPORT RESULTS
# =============================================================================
output_cols = [
    # Identifiers
    'product_id', 'warehouse_id', 'cohort_id', 'sku', 'brand', 'cat', 'stocks',
    # Pricing data
    'current_price', 'wac_p', 'new_price',
    'target_margin', 'min_boundary',
    # Performance data
    'uth_qty', 'uth_retailers',
    'p80_daily_240d', 'p70_daily_retailers_240d', 'avg_uth_pct',
    'sku_disc_cntrb_uth', 't1_cntrb_uth', 't2_cntrb_uth', 't3_cntrb_uth',
    'uth_qty_target', 'uth_retailer_target', 'qty_ratio', 'retailer_ratio', 'uth_status',
    'doh', 'mtd_qty',
    # Cart rules
    'price_action', 'current_cart_rule', 'new_cart_rule',
    # SKU Discount fields
    'activate_sku_discount', 'active_sku_disc_pct', 'has_active_sku_discount',
    # QD fields (for qd_handler)
    'activate_qd', 'keep_qd_tiers', 'has_active_qd',
    'qd_tier_1_qty', 'qd_tier_1_disc_pct',
    'qd_tier_2_qty', 'qd_tier_2_disc_pct',
    'qd_tier_3_qty', 'qd_tier_3_disc_pct',
    # Market margins (for handlers to convert to prices)
    'below_market', 'market_min', 'market_25', 'market_50',
    'market_75', 'market_max', 'above_market',
    # Margin tiers (for handlers to convert to prices)
    'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4',
    'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2',
    # Action tracking
    'removed_discount', 'removed_discount_cntrb',
    'price_reductions_today', 'action_reason'
]

# Filter to existing columns
output_cols = [c for c in output_cols if c in df_results.columns]

# Drop duplicates before saving
df_output = df_results[output_cols].drop_duplicates(subset=['product_id', 'warehouse_id'], keep='first')
df_output.to_excel(OUTPUT_FILE, index=False)
print(f"\n✅ Results exported to: {OUTPUT_FILE}")
print(f"Total records: {len(df_output)} (after removing {len(df_results) - len(df_output)} duplicates)")



✅ Results exported to: module_3_output_20260206_2306.xlsx
Total records: 28147 (after removing 0 duplicates)


In [17]:
# =============================================================================
# PUSH CART RULES & PRICES
# =============================================================================
# Push cart rules FIRST, then prices
# If cart rules fail for certain cohorts, skip those cohorts for prices

%run push_cart_rules_handler.ipynb
%run push_prices_handler.ipynb
pus = get_packing_units()

# ⚠️ MODE CONFIGURATION:
# - 'testing' (default): Prepare files but DON'T upload to API
# - 'live': Prepare files AND upload to MaxAB API
PUSH_MODE = 'live'  # Change to 'live' when ready to push

# =============================================================================
# STEP 1: Push Cart Rules First
# =============================================================================
print("\n" + "="*70)
print("STEP 1: PUSHING CART RULES")
print("="*70)

cart_result = push_cart_rules(df_output, pus, source_module='module_3', mode=PUSH_MODE)

print(f"\n{'='*60}")
print("CART RULES RESULT")
print(f"{'='*60}")
print(f"Mode: {cart_result['mode']}")
print(f"Cart rule changes: {cart_result['cart_rule_changes']}")
print(f"Pushed: {cart_result['pushed']}")
print(f"Failed: {cart_result['failed']}")
if cart_result['failed_cohorts']:
    print(f"⚠️ Failed cohorts: {cart_result['failed_cohorts']}")

# =============================================================================
# STEP 2: Push Prices (skip failed cohorts)
# =============================================================================
print("\n" + "="*70)
print("STEP 2: PUSHING PRICES")
print("="*70)

# Get failed cohorts from cart rules to skip in price push
failed_cohorts = cart_result.get('failed_cohorts', [])

# Call push_prices with the results, skipping failed cohorts
push_result = push_prices(df_output, pus, source_module='module_3', mode=PUSH_MODE, skip_cohorts=failed_cohorts)

print(f"\n{'='*60}")
print("PRICES RESULT")
print(f"{'='*60}")
print(f"Mode: {push_result['mode']}")
print(f"Source: {push_result['source_module']}")
print(f"Timestamp: {push_result['timestamp']}")
print(f"Total received: {push_result['total_received']}")
print(f"Price changes: {push_result['price_changes']}")
print(f"Pushed: {push_result['pushed']}")
print(f"Skipped: {push_result['skipped']}")
print(f"Failed: {push_result['failed']}")
if push_result.get('skipped_cohorts'):
    print(f"⚠️ Skipped cohorts (cart rules failed): {push_result['skipped_cohorts']}")


Push Cart Rules Handler loaded at 2026-02-06 23:07:52 Cairo time
✓ API credentials loaded successfully


Push Prices Handler loaded at 2026-02-06 23:07:52 Cairo time
✓ API credentials loaded successfully


✓ Google Sheets client initialized
Fetching packing_units ...


  Loaded 35105 records

STEP 1: PUSHING CART RULES

🚀 MODE: LIVE
   Files will be prepared AND uploaded to API

PUSH CART RULES - Source: module_3
Total received: 28147
Cart rule changes to push: 8006
Skipped (no change): 20141

Cart rule changes summary:
  Increases: 7148
  Decreases: 858

📋 Prepared 9940 packing unit cart rules

Sample cart rule adjustments (showing products with multiple PUs):
 product_id  basic_unit_count  final_cart_rule  final_pu_cart_rule
          3                 1               20                  20
          3                 1                6                   6
          3                 1               16                  16
          3                 1               36                  36
          3                 1               74                  74
          3                 1               55                  55
          3                 1               19                  19
          9                 1                5                  

  Saved: uploads/module_3_cart_rules_701.xlsx (2118 rows)


...

  Split into 1 chunks (size: 4000)


  Saving chunks:   0%|          | 0/1 [00:00<?, ?it/s]

  Saving chunks: 100%|██████████| 1/1 [00:00<00:00, 59.65it/s]

  Uploading...


    ✓ Chunk 1 uploaded successfully

🚀 UPLOAD COMPLETE
Mode: live
Total prepared: 3961
Total failed: 0

PRICES RESULT
Mode: live
Source: module_3
Timestamp: 2026-02-06 23:08:25
Total received: 28147
Price changes: 3025
Pushed: 3961
Skipped: 25122
Failed: 0


In [18]:
# =============================================================================
# STEP 3: PROCESS SKU DISCOUNTS
# =============================================================================
# This step handles SKU discounts for SKUs that need them based on UTH performance.
# Market data has already been refreshed, so we pass the df_output directly.

print("\n" + "="*70)
print("STEP 3: PROCESSING SKU DISCOUNTS")
print("="*70)

%run sku_discount_handler.ipynb

# Filter to SKUs that need SKU discount
df_sku_discount = df_results[df_results['activate_sku_discount'] == True].copy()
print(f"SKUs needing SKU discount: {len(df_sku_discount)}")

# Merge market margins and margin tiers from df (not in df_results)
sku_discount_extra_cols = [
    'product_id', 'warehouse_id',
    # Market margins
    'below_market', 'market_min', 'market_25', 'market_50', 
    'market_75', 'market_max', 'above_market',
    # Margin tiers
    'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
    'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2',
    # Other needed columns
    'doh', 'zero_demand', 'target_margin', 'min_boundary', 'active_sku_disc_pct'
]
# Filter to columns that exist in df
sku_discount_extra_cols = [c for c in sku_discount_extra_cols if c in df.columns]

# Merge the extra columns from df
df_sku_discount = df_sku_discount.merge(
    df[sku_discount_extra_cols].drop_duplicates(subset=['product_id', 'warehouse_id']),
    on=['product_id', 'warehouse_id'],
    how='left',
    suffixes=('', '_from_df')
)
print(f"  Merged market margins and margin tiers from df")

if len(df_sku_discount) > 0:
    sku_discount_result = process_sku_discounts(df_sku_discount, mode=PUSH_MODE)
    
    print(f"\n{'='*60}")
    print("SKU DISCOUNT RESULT")
    print(f"{'='*60}")
    print(f"Mode: {sku_discount_result['mode']}")
    print(f"Total input: {sku_discount_result['total_input']}")
    print(f"SKUs to activate: {sku_discount_result['to_activate']}")
    print(f"Deactivated: {sku_discount_result['deactivated']}")
    print(f"Created: {sku_discount_result['created']}")
    print(f"Failed: {sku_discount_result['failed']}")
else:
    print("No SKUs need SKU discounts")

# =============================================================================
# STEP 4: PROCESSING QUANTITY DISCOUNTS (QD)
# =============================================================================
# This step handles QD adjustments for SKUs flagged by the action engine.
# Only processes SKUs where activate_qd=True and uses keep_qd_tiers to determine
# which tiers to maintain.

print("\n" + "="*70)
print("STEP 4: PROCESSING QUANTITY DISCOUNTS")
print("="*70)

%run qd_handler.ipynb

# Filter to SKUs that need QD processing
df_qd = df_results[df_results['activate_qd'] == True].copy()
print(f"SKUs needing QD processing: {len(df_qd)}")

# Required columns for QD handler
# Include all data needed for tier quantity and price calculations
qd_columns = [
    # Identifiers
    'product_id', 'warehouse_id', 'cohort_id', 'sku', 'brand', 'cat',
    # Pricing data
    'wac_p', 'current_price', 'new_price', 'target_margin', 'min_boundary',
    # Cart rules
    'current_cart_rule', 'new_cart_rule',
    # Market margins (to be converted to prices)
    'below_market', 'market_min', 'market_25', 'market_50',
    'market_75', 'market_max', 'above_market',
    # Margin tiers (to be converted to prices)
    'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 'margin_tier_4',
    'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2',
    # Performance data (for top SKU selection)
    'mtd_qty',
    # QD configuration
    'keep_qd_tiers'
]
# Filter to columns that exist in df_results
qd_columns = [c for c in qd_columns if c in df_results.columns]
df_qd = df_qd[qd_columns].copy()

if len(df_qd) > 0:
    qd_result = process_qd(df_qd, False)
    
    print(f"\n{'='*60}")
    print("QD PROCESSING RESULT")
    print(f"{'='*60}")
    print(f"Mode: {qd_result['mode']}")
    print(f"Total input: {qd_result['total_input']}")
    print(f"Processed: {qd_result['processed']}")
    print(f"Failed: {qd_result['failed']}")
else:
    print("No SKUs need QD processing")

# =============================================================================
# FINAL SUMMARY
# =============================================================================
print("\n" + "="*70)
print("MODULE 3 EXECUTION COMPLETE")
print("="*70)
print(f"Total SKUs processed: {len(df_output)}")
print(f"Price changes: {(df_output['new_price'] != df_output['current_price']).sum()}")
print(f"Cart rule changes: {(df_output['new_cart_rule'] != df_output['current_cart_rule']).sum()}")
print(f"SKUs with SKU discount: {df_output['activate_sku_discount'].sum()}")
print(f"SKUs with QD: {df_output['activate_qd'].sum()}")
print(f"Output saved to: {OUTPUT_FILE}")



STEP 3: PROCESSING SKU DISCOUNTS


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


SKU Discount Handler loaded at 2026-02-06 23:09:19 Cairo time
Excluded categories: ['كروت شحن']
Excluded brands: ['فيوري', 'العروسة']
AWS & API functions defined ✓
✓ API credentials loaded successfully


Snowflake timezone: America/Los_Angeles
Function 1: deactivate_active_sku_discounts() defined ✓


/home/ec2-user/.Renviron
/home/ec2-user/service_account_key.json


...

    ✓ Cohort 704: 26 rules uploaded


    ✓ Cohort 1123: 4 rules uploaded


    ✓ Cohort 1124: 4 rules uploaded


    ✓ Cohort 1125: 4 rules uploaded


    ✓ Cohort 1126: 1 rules uploaded

  Cart Rules Result:
    Cohorts updated: 9
    Cohorts failed: 0

QD HANDLER - SUMMARY
Mode: LIVE
Total SKUs in input: 620
SKUs with valid T1 & T2 prices: 217
SKUs with valid T3 prices: 70
SKUs after keep_qd_tiers & 400 tier limit: 202
Total tier entries: 466
Valid QD configs: 202
QD found active: 12
QD deactivated: 12
QD created: 202
QD creation failed: 0
Cart rules updated: 188 products

QD PROCESSING RESULT
Mode: live
Total input: 620
Processed: 202
Failed: 0

MODULE 3 EXECUTION COMPLETE
Total SKUs processed: 28147
Price changes: 28076
Cart rule changes: 24726
SKUs with SKU discount: 16700
SKUs with QD: 620
Output saved to: module_3_output_20260206_2306.xlsx


In [19]:
# =============================================================================
# UPLOAD RESULTS TO SNOWFLAKE AND SEND SLACK NOTIFICATION
# =============================================================================
from common_functions import upload_dataframe_to_snowflake, send_text_slack

# Add created_at as TIMESTAMP (module runs multiple times per day)
df_output = df_output.drop(columns=['keep_qd_tiers'], errors='ignore')
df_output['keep_qd_tiers'] = np.nan
df_output['created_at'] = datetime.now(CAIRO_TZ).replace(second=0, microsecond=0)

# Upload to Snowflake
print("\n" + "="*60)
print("UPLOADING RESULTS TO SNOWFLAKE")
print("="*60)

upload_status = upload_dataframe_to_snowflake(
    "Egypt", 
    df_output, 
    "MATERIALIZED_VIEWS", 
    "pricing_periodic_push", 
    "append", 
    auto_create_table=True, 
    conn=None
)

# Prepare status variables
prices_pushed = push_result.get('pushed', 0) if 'push_result' in dir() else 0
prices_failed = push_result.get('failed', 0) if 'push_result' in dir() else 0
cart_rules_pushed = cart_result.get('pushed', 0) if 'cart_result' in dir() else 0
cart_rules_failed = cart_result.get('failed', 0) if 'cart_result' in dir() else 0

# SKU discount status
sku_disc_processed = len(df_sku_discount) if 'df_sku_discount' in dir() else 0

# QD status
qd_processed = qd_result.get('processed', 0) if 'qd_result' in dir() and qd_result else 0
qd_failed = qd_result.get('failed', 0) if 'qd_result' in dir() and qd_result else 0
df_output.columns = df_output.columns.str.lower()
if upload_status:
    slack_message = f"""✅ *Module 3 - Periodic Actions Completed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Completed at: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
🔧 Mode: {PUSH_MODE.upper()}

📊 *Results:*
• Total SKUs processed: {len(df_output):,}
• Price changes: {(df_output['new_price'] != df_output['current_price']).sum():,}
• Induced DOH prices: {(df_output['price_action'] == 'induced_doh_reduction').sum():,}
• Cart rule changes: {(df_output['new_cart_rule'] != df_output['current_cart_rule']).sum():,}

📤 *Push Status:*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed
• 🏷️ SKU Discounts: {sku_disc_processed} processed
• 📦 Quantity Discounts: ✅ {qd_processed} processed | ❌ {qd_failed} failed

🗄️ Results uploaded to: MATERIALIZED_VIEWS.pricing_periodic_push"""
    
    send_text_slack('new-pricing-logic', slack_message)
    print("✅ Slack notification sent!")
    print(f"✅ {len(df_output)} records uploaded to Snowflake")
else:
    error_message = f"""❌ *Module 3 - Periodic Actions Failed*

📅 Date: {datetime.now(CAIRO_TZ).strftime('%Y-%m-%d')}
⏰ Failed at: {datetime.now(CAIRO_TZ).strftime('%H:%M:%S')} Cairo time
⚠️ Upload to Snowflake failed - please check logs

📤 *Push Status (before upload failure):*
• 💰 Prices: ✅ {prices_pushed} pushed | ❌ {prices_failed} failed
• 🛒 Cart Rules: ✅ {cart_rules_pushed} pushed | ❌ {cart_rules_failed} failed
• 🏷️ SKU Discounts: {sku_disc_processed} processed
• 📦 Quantity Discounts: ✅ {qd_processed} processed | ❌ {qd_failed} failed"""
    
    send_text_slack('new-pricing-logic', error_message)
    print("❌ Error notification sent to Slack!")



UPLOADING RESULTS TO SNOWFLAKE


/home/ec2-user/service_account_key.json


/home/ec2-user/SageMaker/Pricing Runs/Prediction_Scripts_2/Happy_hour/git/Mustafa/Pricing Logic/modules/../common_functions.py:698: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, _, _, _ = write_pandas(


/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/slack/deprecation.py:14: UserWarning: slack package is deprecated. Please use slack_sdk.web/webhook/rtm package instead. For more info, go to https://docs.slack.dev/tools/python-slack-sdk/v3-migration/
  warnings.warn(message)


/home/ec2-user/service_account_key.json


Message Sent
✅ Slack notification sent!
✅ 28147 records uploaded to Snowflake
